### YOLOv8 demo for CAST

In [ ]:
# use this block to download requirements into virtual environment

<u>Import statements<u>

In [1]:
import os
from ultralytics import YOLO
import cv2
import torch

<u>Importing training dataset for demo</u>

For this demo, we will be training a YOLOv8 on African wildlife imagery pulled from https://ultralytics.com/assets/african-wildlife.zip.....

API KEY:

https://public.roboflow.com/ds/u2iU8ZerKB?key=xuwb9jGSVi